In [1]:
import pandas as pd
import numpy as np
import scipy.stats as sps 
import matplotlib.pyplot as plt 
import seaborn as sns
import sklearn as skl
from sklearn import preprocessing 
%matplotlib inline

In [2]:
%%time
data=pd.read_csv('after_fill_missing.csv', index_col=0)

CPU times: user 210 ms, sys: 40.6 ms, total: 251 ms
Wall time: 252 ms


In [3]:
from datetime import datetime
data['Date'] = pd.to_datetime(pd.Series(data['Date']), infer_datetime_format=True)

In [5]:
%%time
#Merchnum
for groupbyvar in ['Merchnum']:
    data_sorted = data.sort_values(by = [groupbyvar, 'Date'])
    data_sorted_index = data_sorted.set_index('Date')
    for agg in ['mean', 'max', 'median', 'sum', 'count']: 
        for days in [1,2,4,8,15,31]:
            data_sorted[agg + '_' + groupbyvar + "_" + str(days-1)+'d']=getattr(data_sorted_index.groupby(groupbyvar)['Amount'].rolling(str(days)+'d'),agg)().values
            data_sorted['Actual/' + agg + "_" + groupbyvar + "_" + str(days-1)+'d'] = data_sorted['Amount']/data_sorted[agg + '_' + groupbyvar + "_" + str(days-1)+'d']

CPU times: user 2min 19s, sys: 1.24 s, total: 2min 21s
Wall time: 2min 21s


In [10]:
%%time
#Cardnum
for groupbyvar in ['Cardnum']:
    data_sorted = data_sorted.sort_values(by = [groupbyvar, 'Date'])
    data_sorted_index = data_sorted.set_index('Date')
    for agg in ['mean', 'max', 'median', 'sum', 'count']: 
        for days in [1,2,4,8,15,31]:
            data_sorted[agg + '_' + groupbyvar + "_" + str(days-1)+'d']=getattr(data_sorted_index.groupby(groupbyvar)['Amount'].rolling(str(days)+'d'),agg)().values
            data_sorted['Actual/' + agg + "_" + groupbyvar + "_" + str(days-1)+'d'] = data_sorted['Amount']/data_sorted[agg + '_' + groupbyvar + "_" + str(days-1)+'d']

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: 'Date' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  This is separate from the ipykernel package so we can avoid doing imports until


CPU times: user 19.5 s, sys: 642 ms, total: 20.1 s
Wall time: 19.9 s


In [6]:
%%time
#Cardnum and Merchnum, Cardnum and Zip, Cardnum and state
for groupbyvar in ['Merchnum', 'Merch zip', 'Merch state']:
    data_sorted = data_sorted.sort_values(by = ['Cardnum',groupbyvar, 'Date'])
    data_sorted_index = data_sorted.set_index('Date')
    for agg in ['mean', 'max', 'median', 'sum', 'count']: 
        for days in [1,2,4,8,15,31]:
            data_sorted[agg + '_' + "Cardnum_" + groupbyvar + "_" + str(days-1)+'d'] = getattr(data_sorted_index.groupby(['Cardnum',groupbyvar])['Amount'].rolling(str(days)+'d'),agg)().values
            data_sorted['Actual/' + agg + "_" + "Cardnum_" + groupbyvar + "_" + str(days-1)+'d'] = data_sorted['Amount']/data_sorted[agg + '_' + "Cardnum_" + groupbyvar + "_" + str(days-1)+'d']            


CPU times: user 17min 29s, sys: 20.4 s, total: 17min 50s
Wall time: 18min


In [7]:
%%time
#Create days since variables
for groupbyvar in [['Cardnum'], ['Merchnum'], ['Cardnum', 'Merchnum'], ['Cardnum', 'Merch zip'], ['Cardnum', 'Merch state']]:
    sortCols = groupbyvar[:]
    sortCols.append('Date')
    data_sorted = data_sorted.sort_values(by = sortCols)
    if len(groupbyvar) == 1:
        data_sorted['Days_since_per_' + groupbyvar[0]] = data_sorted.groupby(groupbyvar)['Date'].apply(lambda x: (x - x.shift(1)).astype('timedelta64[D]')).fillna(365).values 
    else:
        data_sorted['Days_since_per_Cardnum_' + groupbyvar[1]] = data_sorted.groupby(groupbyvar)['Date'].apply(lambda x: (x -x.shift(1)).astype('timedelta64[D]')).fillna(365).values 


CPU times: user 1min 13s, sys: 2.15 s, total: 1min 15s
Wall time: 1min 14s


In [ ]:
###!!!!!!!!!check the index for the head of dataset first and need to set date as index

In [8]:
data_sorted.index = data_sorted['Date']

In [ ]:
data_sorted

In [9]:
%%time
##Frequency

name=['Card_Fre','Merch_Fre','Card/Merch_Fre','Card/Zip_Fre','Card/State_Fre']
df={}
for group in [['Cardnum'], ['Merchnum'], ['Cardnum', 'Merchnum'], ['Cardnum', 'Merch zip'], ['Cardnum', 'Merch state']]:
    for days in [1,2,4,8,15,31]:
        for n in name:
            df[n]=pd.DataFrame(data_sorted.groupby(group)['Amount'].rolling(str(days)+'d').count().reset_index())
            df[n]=df[n].groupby(group)['Amount'].transform(max).reset_index()
            data_sorted[str(n)+str(days-1)]=df[n].iloc[:,-1].values  

CPU times: user 20min 31s, sys: 29.4 s, total: 21min 1s
Wall time: 21min 13s


In [14]:
%%time
##number of transaction Velocity

name_vel=['Card_Fre','Merch_Fre']
for days in [1,2]:
    for days1 in [8,15,31]:
        for n in name_vel:
            data_sorted[str(n)+str(days-1)+'/'+str(days1-1)+'Vel']=data_sorted[str(n)+str(days-1)]/(data_sorted[str(n)+str(days1-1)]/(days1-1))


CPU times: user 27.3 ms, sys: 5.62 ms, total: 32.9 ms
Wall time: 24.5 ms


In [12]:
%%time
##Amount Velocity

name_vel=['Cardnum', 'Merchnum']
for days in [1,2]:
    for days1 in [8,15,31]:
        for n in name_vel:
            data_sorted['AMT'+str(n)+str(days-1)+'/'+str(days1-1)+'Vel']=data_sorted['sum' + '_' + n + "_" + str(days-1)+'d']/(data_sorted['sum' + '_' + n + "_" + str(days1-1)+'d']/(days1-1))
         

CPU times: user 40.1 ms, sys: 9.61 ms, total: 49.8 ms
Wall time: 34.3 ms


In [16]:
data_sorted['Merchnum'].value_counts()

930090121224     11525
5509006296254     2131
9900020006406     1714
602608969534      1093
4353000719908     1022
410000971343       982
9918000409955      958
5725000466504      872
9108234610000      817
602608969138       783
4503082476300      748
2094206450000      590
4063000739258      570
2094330000009      534
6920602000804      523
8834000695412      479
4591200814444      463
602608969284       442
9900000037306      435
6859858914649      432
806967335334       431
2603800006005      421
6170666166         392
7725000069608      362
809946579333       345
9108234610006      344
602608964157       309
08-0963129334      307
9108347680000      286
9900000036906      285
                 ...  
6072365000066        1
2722000934765        1
900009056002         1
4800000622540        1
35966257330          1
60046726667          1
8057000446202        1
700989109            1
6070011930005        1
7739600093614        1
6129602005583        1
8008770048906        1
59000000079

In [14]:
data_sorted.to_csv("final data.csv")

In [18]:
a=0
for i in data_sorted['Merchnum']:
    if str(i)== 0:
        a+=1
a

0